In [1]:
import os
import sys

from pathlib import Path

import pandas as pd
import wandb


sys.path.append("..")
from otc.metrics.metrics import effective_spread


In [ ]:
os.environ["GCLOUD_PROJECT"] = "flowing-mantis-239216"

run = wandb.init(project="thesis", entity="fbv")

dataset = "fbv/thesis/train_val_test_ultra:v0"
results = "fbv/thesis/results_classical_clf:v0"
fname_dataset = "test_set_extended_20"
fname_results = "results_classical_clf_ise"


# load unscaled data
artifact = run.use_artifact(dataset) # type: ignore
data_dir = artifact.download()

# load results
artifact = run.use_artifact(results) # type: ignore
results_dir = artifact.download()


In [ ]:
columns = ["TRADE_PRICE", "bid_ex", "ask_ex", "buy_sell"]

eval_data = pd.read_parquet(
    Path(data_dir, fname_dataset), engine="fastparquet", columns=columns
)
results_data = pd.read_parquet(
    Path(results_dir, fname_results), engine="fastparquet"
)

results_col = [results_data.columns.tolist(), "buy_sell"]

composed_data = pd.concat([eval_data, results_data], axis=1)
composed_data["mid"] = (composed_data["bid_ex"] + composed_data["ask_ex"]) / 2

del eval_data, results_data


In [ ]:
results = []
for col in results_col:
    results.append(
        {
            f"eff_{composed_data[col]}": effective_spread(
                composed_data[col], composed_data["TRADE_PRICE"], composed_data["mid"] # type: ignore
            )
        }
    )


In [ ]:
results_df = pd.DataFrame(results)